# Main idea

A script that generates an agenda from the open data of the Chamber of Deputies (https://dadosabertos.camara.leg.br/swagger/api.html) given a list of events of interest.

**Importing libraries**

In [167]:
import requests
import json
import datetime
import pandas as pd
import time
import os
import shutil

**Create folder to alocate json files from events and agenda**

In [168]:
os.mkdir('Pautas')
os.mkdir('Eventos')
os.mkdir('Dia')

**Generating list of events of interest** 

Here the list comes from an excel table.

In [169]:
df = pd.read_excel("Lista Projetos Camara.xlsx") # Read excel file
ListaBacklog = df['Lista'].tolist() # Make a list from the dataframe

**Get list of events ID´s through URL**

In [172]:
IdSol = [] #List of ID´s of events 
for i in range(8): #Iterate over a week to get the Events ID´s
    Data = str((datetime.datetime.today() + datetime.timedelta(days=i)).date()).split('-')
    AnoR = Data[0]
    MesR = Data[1]
    DiaR = Data[2]
    # Call URL to get json file
    URL = 'https://dadosabertos.camara.leg.br/api/v2/eventos?dataInicio={0}-{1}-{2}&dataFim={0}-{1}-{2}&ordem=ASC&ordenarPor=dataHoraInicio'.format(AnoR,MesR,DiaR)
    response = requests.get(URL)
    # Open Json file
    with open('Dia/EventoDia{}.json'.format(Data), 'wb') as file:
        file.write(response.content)
    file.close()
    with open('Dia/EventoDia{}.json'.format(Data), 'r', encoding="utf8") as file:
        DadoDia = json.load(file)
    file.close()
    # Save Event´s ID
    for D in DadoDia['dados']:
        IdSol.append(D['id'])

**Dict entries needed**

In [173]:
Output = {'uriDeputados': [],
          'uriConvidados': [],
          'fases': [],
          'id': [],
          'uri': [],
          'dataHoraInicio': [],
          'dataHoraFim': [],
          'situacao': [],
          'descricaoTipo': [],
          'descricao': [],
          'localExterno': [],
          'localCamaraNome': [],
          'localCamaraPredio': [],
          'localCamaraSala': [],
          'localCamaraAndar': [],
          'orgaosId': [],
          'orgaosUri': [],
          'orgaosSigla': [],
          'orgaosNome': [],
          'orgaosApelido': [],
          'orgaosCodTipo': [],
          'orgaosTipoOrgao': [],
          'nomePublicacao': [],
          'urlDocumentoPauta': [],
          'urlRegistro': [],
          'regime': [],
          'codRegime': [],
          'ordem': [],
          'idProp': [],
          'uriProp': [],
          'siglaTipo': [],
          'codTipo': [],
          'numero': [],
          'ano': [],
          'ementa': [],
          'situacaoItem': [],
          'uriVotacao': []
          }

**Iterate over events and look from propositions that are in the list**

In [174]:
for Id in IdSol:
    URL = 'https://dadosabertos.camara.leg.br/api/v2/eventos/{}/pauta'.format(Id)
    response = requests.get(URL)
    with open('Pautas/Pauta{}.json'.format(Id), 'wb') as file:
        file.write(response.content)
    file.close()
    with open('Pautas/Pauta{}.json'.format(Id), 'r', encoding="utf8") as file:
        DadoPauta = json.load(file)
    file.close()
    for D in DadoPauta['dados']:
        if int(D['proposicao_']['id']) in ListaBacklog:
            # Dados Pauta
            Output['regime'].append(D['regime'])
            Output['codRegime'].append(D['codRegime'])
            Output['ordem'].append(D['ordem'])
            Output['idProp'].append(D['proposicao_']['id'])
            Output['uriProp'].append(D['proposicao_']['uri'])
            Output['siglaTipo'].append(D['proposicao_']['siglaTipo'])
            Output['codTipo'].append(D['proposicao_']['codTipo'])
            Output['numero'].append(D['proposicao_']['numero'])
            Output['ano'].append(D['proposicao_']['ano'])
            Output['ementa'].append(D['proposicao_']['ementa'])            
            Output['situacaoItem'].append(D['situacaoItem'])
            Output['uriVotacao'].append(D['uriVotacao'])

            # Dados Evento
            URL = 'https://dadosabertos.camara.leg.br/api/v2/eventos/{}'.format(Id)
            response = requests.get(URL)
            with open('Eventos/Evento{}.json'.format(Id), 'wb') as file:
                file.write(response.content)
            file.close()
            with open('Eventos/Evento{}.json'.format(Id), 'r', encoding="utf8") as file:
                DadoEvento = json.load(file)
            file.close()
            Output['uriDeputados'].append(DadoEvento['dados']['uriDeputados'])
            Output['uriConvidados'].append(DadoEvento['dados']['uriConvidados'])
            Output['fases'].append(DadoEvento['dados']['fases'])
            Output['id'].append(DadoEvento['dados']['id'])
            Output['uri'].append(DadoEvento['dados']['uri'])
            Output['dataHoraInicio'].append(DadoEvento['dados']['dataHoraInicio'])
            Output['dataHoraFim'].append(DadoEvento['dados']['dataHoraFim'])
            Output['situacao'].append(DadoEvento['dados']['situacao'])
            Output['descricaoTipo'].append(DadoEvento['dados']['descricaoTipo'])
            Output['descricao'].append(DadoEvento['dados']['descricao'])
            Output['localExterno'].append(DadoEvento['dados']['localExterno'])
            Output['localCamaraNome'].append(DadoEvento['dados']['localCamara']['nome'])
            Output['localCamaraPredio'].append(DadoEvento['dados']['localCamara']['predio'])
            Output['localCamaraSala'].append(DadoEvento['dados']['localCamara']['sala'])
            Output['localCamaraAndar'].append(DadoEvento['dados']['localCamara']['andar'])
            Output['orgaosId'].append(DadoEvento['dados']['orgaos'][0]['id'])
            Output['orgaosUri'].append(DadoEvento['dados']['orgaos'][0]['uri'])
            Output['orgaosSigla'].append(DadoEvento['dados']['orgaos'][0]['sigla'])
            Output['orgaosNome'].append(DadoEvento['dados']['orgaos'][0]['nome'])
            Output['orgaosApelido'].append(DadoEvento['dados']['orgaos'][0]['apelido'])
            Output['orgaosCodTipo'].append(DadoEvento['dados']['orgaos'][0]['codTipoOrgao'])
            Output['orgaosTipoOrgao'].append(DadoEvento['dados']['orgaos'][0]['tipoOrgao'])
            Output['nomePublicacao'].append(DadoEvento['dados']['orgaos'][0]['nomePublicacao'])
            Output['urlDocumentoPauta'].append(DadoEvento['dados']['urlDocumentoPauta'])
            Output['urlRegistro'].append(DadoEvento['dados']['urlRegistro'])

**Extract data as excel file**

In [175]:
for Key in Output:
    if Output[Key].count(None) > 0:
        Output[Key][Output[Key].index(None)] = '-'
Planilha = pd.DataFrame.from_dict(Output)
Planilha.to_excel('Agenda Camara.xlsx')

**Delete de folders and json files scratched**

In [176]:
shutil.rmtree('Eventos')
shutil.rmtree('Pautas')
shutil.rmtree('Dia')